![hand_pose](img/lemma_hand_pose.jpg)

Reconocimiento de caracteres del lenguaje de señas peruano, los cuales se detallarán posteriormente.

Consideraciones previas:
- No se reconocerán caracteres que requieran movimientos. 
- No se Reconocerán caracteres que requieran el uso de ambas manos.
- Se creará un dataset sólo para el reconocimiento de la mano Derecha.
- Algunos caracteres compartirán 2 representaciones, ya que la misma pose reprsenta un número y/o un número.

Los caracteres reconocidos por el modelo:
- Letras: A,B,C,D,E,F,G,H,I,K,L,M,N,O,P,Q,R,S,T,U,V,W,X,Y # No se reconoce la J, Ñ ni la Z (requieren movimientos).
- Números: 0,1,2,3,4,5,6,7,8,9 # 10 números básicos.

Fuente: 
- Perú │ Ministerio de Educación.
- Libro: Lengua de señas peruana : guía para el aprendizaje de la lengua de señas peruana, vocabulario básico (2015). 
- Enlace: https://repositorio.minedu.gob.pe/handle/20.500.12799/5545

### Modelo de reconocimiento dacticológico

In [1]:
%pip install mediapipe opencv-python

Note: you may need to restart the kernel to use updated packages.


In [2]:
import mediapipe as mp
import cv2
import numpy as np
import pickle 
import pandas as pd
import os
from collections import Counter

In [3]:
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands

In [4]:
modelos = {
    'alphabet': 'Modelos/modelo_letters.pkl',
    'numeric': 'Modelos/modelo_numbers.pkl'
}

In [5]:
# Inicialización de la cámara
cap = cv2.VideoCapture(0)

# Cargar y ajustar el logo
logo = cv2.imread('img/log_urp_completo.png', cv2.IMREAD_UNCHANGED)
logo = cv2.resize(logo, (165, 50))

# Variables globales
clases_finales = []
clasificando = False
clase_text = ""
confianza_text = ""
hand_label_text = ""

def cargar_modelo(nombre_modelo):
    """Carga del modelo."""
    with open(modelos[nombre_modelo], 'rb') as f:
        return pickle.load(f)

def reflect_landmarks(landmarks):
    """Refleja los landmarks de la mano para la mano izquierda."""
    for landmark in landmarks:
        landmark.x = 1 - landmark.x
    return landmarks


def predict_hand_pose(hand_landmarks, model):
    """Predice la clase y la confianza de una pose de mano."""
    pose_row = list(np.array([[landmark.x, landmark.y, landmark.z] for landmark in hand_landmarks]).flatten())
    X = pd.DataFrame([pose_row])
    clase = model.predict(X)[0]
    confianza = model.predict_proba(X)[0]
    return clase, confianza


def draw_ui(image, clase_text, confianza_text, hand_label_text, clases_finales):
    """Dibuja la interfaz gráfica en la imagen."""
    height, width, _ = image.shape
    cv2.rectangle(image, (0, 0), (width, 83), (255, 255, 255), -1)

    # Agregar logo
    image[4:54, 9:174] = logo

    # Texto estático
    cv2.putText(image, 'Clase:', (190, 35), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
    cv2.putText(image, 'Exactitud:', (285, 35), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
    cv2.putText(image, 'Mano:', (420, 35), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)

    # Texto dinámico
    cv2.putText(image, clase_text, (245, 35), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (61, 97, 20), 2, cv2.LINE_AA)
    cv2.putText(image, confianza_text, (370, 35), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (61, 97, 20), 2, cv2.LINE_AA)
    cv2.putText(image, hand_label_text, (470, 35), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (61, 97, 20), 2, cv2.LINE_AA)

    # Clases acumuladas
    clases_text = f"... , {', '.join(clases_finales[-8:])}" if len(clases_finales) > 18 else f"{', '.join(clases_finales)}"
    cv2.putText(image, f"Clases acumuladas: {clases_text}", (10, 73), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (61, 97, 20), 1, cv2.LINE_AA)

    return image

modelo_actual = cargar_modelo('alphabet')
modelos_disponibles = list(modelos.keys())
modelo_index = 0

with mp_hands.Hands(max_num_hands=1, min_detection_confidence=0.8, min_tracking_confidence=0.4) as hands:
    while cap.isOpened():
        ret, frame = cap.read()

        if not ret:
            break

        frame = cv2.flip(frame, 1)
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        results = hands.process(image)
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # Clasificación solo si se activa
        if results.multi_hand_landmarks and clasificando:
            for hand_landmarks, handedness in zip(results.multi_hand_landmarks, results.multi_handedness):
                hand_label = handedness.classification[0].label
                hand_label_text = "Derecha" if hand_label == "Right" else "Izquierda"

                mp_drawing.draw_landmarks(image, hand_landmarks, mp_hands.HAND_CONNECTIONS,
                    mp_drawing.DrawingSpec(color=(61, 97, 20), thickness=2, circle_radius=4),
                    mp_drawing.DrawingSpec(color=(80, 44, 121), thickness=2, circle_radius=2))

                # Reflejar si es mano izquierda
                landmarks = reflect_landmarks(hand_landmarks.landmark) if hand_label == 'Left' else hand_landmarks.landmark
                clase, confianza = predict_hand_pose(landmarks, modelo_actual)

                clase_text = clase.split(' ')[0]  # Solo la primera palabra (como la clase)
                confianza_text = str(round(confianza[np.argmax(confianza)], 2))

        else:
            clase_text = ""
            confianza_text = ""
            hand_label_text = ""

        # Dibuja la UI en la imagen
        image = draw_ui(image, clase_text, confianza_text, hand_label_text, clases_finales)

        # Mostrar la imagen
        cv2.imshow('Deteccion de valores', image)

        key = cv2.waitKey(10) & 0xFF
        if key == ord('q'):  # Cerrar el modelo
            break
        elif key == ord('p'):  # Alternar entre activar y desactivar la clasificación
            clasificando = not clasificando
        elif key == ord('d'):  # Eliminar la última clase almacenada
            if clases_finales:
                clases_finales.pop()
        elif key == ord('b'):  # Eliminar todas las clases almacenadas
            clases_finales.clear()
        elif key == ord('s'):  # Agregar espacio
            if clase_text:
                clases_finales.append(" ")
        elif key == ord('g'):  # Grabar la clase cuando se presiona 'g'
            if clase_text:
                clases_finales.append(clase_text)
        elif key == ord('m'):  # Alternar entre modelos
            modelo_index = (modelo_index + 1) % len(modelos_disponibles)
            modelo_actual = cargar_modelo(modelos_disponibles[modelo_index])
            print(f"Modelo cambiado a '{modelos_disponibles[modelo_index]}'")

cap.release()
cv2.destroyAllWindows()

c:\Users\maste\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\maste\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\maste\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\maste\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\maste\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScal

Modelo cambiado a 'numeric'


c:\Users\maste\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\maste\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\maste\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\maste\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\maste\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScal